In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
from get_args import get_parser
import torch
from transformers import BertTokenizerFast
from dataset import *
from train import *
from model import *
import random
import numpy as np
import torch.nn as nn

In [ ]:
class Args:
    model_name = 'hfl/chinese-roberta-wwm-ext'
    max_length = 128
args = Args()

In [ ]:
model = NERThreeModel.from_pretrained(args.model_name, num_labels=len(id2label),max_length=args.max_length)
model.load_state_dict(torch.load('./ckpts/ner3_epoch_3.pt'))
# model = nn.DataParallel(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [6]:
sent = '机器学习是一种很牛逼的方法。'
tk = BertTokenizerFast.from_pretrained(args.model_name)
inp = tk(
    [list(sent),],
    padding='max_length',
    truncation=True,
    max_length=args.max_length,
    is_split_into_words=True,
    return_tensors='pt'
)
# inp
b_input_ids = inp['input_ids'].to(device)
b_input_mask = inp['attention_mask'].to(device)

output = model(input_ids=b_input_ids,
                attention_mask=b_input_mask,
                labels=None)[0]
output = torch.argmax(output,dim=-1).cpu().numpy()
extracted_terms = []
output = output[0]
range_len = len(output)
output = [id2label[i] for i in output]
# for 
# print(output)
term = ''
previous = None
for i in range(range_len):
    
    if i > len(sent):
        break
    # print(sent[i],output[i])
    if output[i] == 'O':
        pass
    elif output[i] == 'B':
        term = sent[i]
    elif output[i] == 'I':
        if previous=='B' or previous=='I':
            term += sent[i]
    elif output[i] == 'E':
        if previous=='B' or previous=='I':
            term += sent[i]
        extracted_terms.append(term)
    elif output[i] == 'S':
        extracted_terms.append(sent[i])

    previous = output[i]
extracted_terms

['机器学习']